This notebook is a basic tutorial that demonstrates how to make an actor prefab and put it into a simulation using Concordia.

In [ ]:
# @title Imports

from collections.abc import Mapping
import dataclasses
import numpy as np
from IPython import display

import sentence_transformers

from concordia.agents import entity_agent_with_logging
from concordia.associative_memory import basic_associative_memory
from concordia.components import agent as agent_components
from concordia.document import interactive_document

from concordia.language_model.model_client_initialization import ModelClient
from concordia.language_model import openrouter_model

import concordia.prefabs.entity as entity_prefabs
import concordia.prefabs.game_master as game_master_prefabs

from concordia.prefabs.simulation import generic as simulation
from concordia.type_checks import prefab as prefab_lib
from concordia.utils import helper_functions


In [ ]:
# Initialize the client
client = ModelClient()
model = client.model
embedder = client.embedder

In [ ]:
# @title Load prefabs from packages to make the specific palette to use here.

prefabs = {
    **helper_functions.get_package_classes(entity_prefabs),
    **helper_functions.get_package_classes(game_master_prefabs),
}

In [ ]:
#@title Print menu of prefabs

display.display(
    display.Markdown(helper_functions.print_pretty_prefabs(prefabs)))

In [ ]:
DEFAULT_INSTRUCTIONS_COMPONENT_KEY = 'Instructions'
DEFAULT_INSTRUCTIONS_PRE_ACT_LABEL = '\nInstructions'


@dataclasses.dataclass
class MyAgent(prefab_lib.Prefab):
  """A prefab implementing an entity with a minimal set of components."""

  description: str = (
      'An entity that has a minimal set of components and is configurable by'
      ' the user. The initial set of components manage memory, observations,'
      ' and instructions. If goal is specified, the entity will have a goal '
      'constant component.'
  )
  params: Mapping[str, str] = dataclasses.field(
      default_factory=lambda: {
          'name': 'Alice',
      }
  )

  def build(
      self,
      model: openrouter_model,
      memory_bank: basic_associative_memory.AssociativeMemoryBank,
  ) -> entity_agent_with_logging.EntityAgentWithLogging:
    """Build an agent.

    Args:
      model: The language model to use.
      memory_bank: The agent's memory_bank object.

    Returns:
      An entity.
    """

    agent_name = self.params.get('name', 'Bob')

    instructions = agent_components.instructions.Instructions(
          agent_name=agent_name,
          pre_act_label=DEFAULT_INSTRUCTIONS_PRE_ACT_LABEL,
      )

    observation_to_memory = agent_components.observation.ObservationToMemory()

    observation_label = '\nObservation'
    observation = agent_components.observation.LastNObservations(
        history_length=100, pre_act_label=observation_label
    )

    principle = agent_components.question_of_recent_memories.QuestionOfRecentMemories(
        model=model,
        pre_act_label=f'{agent_name} main guiding principle:',
        question=(f'How can {agent_name} exploit the situation for personal '
                  'gain and gratification?'),
        answer_prefix=f'{agent_name} understands that ',
        add_to_memory=False,
    )

    components_of_agent = {
        DEFAULT_INSTRUCTIONS_COMPONENT_KEY: instructions,
        'observation_to_memory': observation_to_memory,
        agent_components.observation.DEFAULT_OBSERVATION_COMPONENT_KEY: (
            observation
        ),
        agent_components.memory.DEFAULT_MEMORY_COMPONENT_KEY: (
            agent_components.memory.AssociativeMemory(memory_bank=memory_bank)
        ),
        'principle': principle,
    }

    component_order = list(components_of_agent.keys())

    act_component = agent_components.concat_act_component.ConcatActComponent(
        model=model,
        component_order=component_order,
    )

    agent = entity_agent_with_logging.EntityAgentWithLogging(
        agent_name=agent_name,
        act_component=act_component,
        context_components=components_of_agent,
    )

    return agent


In [ ]:
prefabs['myagent__Entity'] = MyAgent()

In [ ]:
# @title Generate initial conditions for the simulation

YEAR = 1546
PLACE = 'St Andrews, Scotland'
NUM_STATEMENTS = 20
NAMES_TO_GENERATE = 10

prompt = interactive_document.InteractiveDocument(model)
unparsed_statements = prompt.open_question(
    question=(f"Generate a string of {NUM_STATEMENTS} facts about {PLACE} in "
              f"the year {YEAR}. Write them in present tense. Separate them "
              "with ' *** '."),
    max_tokens=4500,
)
statements = unparsed_statements.split('***')
statements = [s.strip() for s in statements]

for statement in statements:
  print(statement)

unparsed_names = prompt.open_question(
    f"Generate a string of {NAMES_TO_GENERATE} names appropriate for this "
    "time and place. Include surnames. Separate them with ' *** '")
names = unparsed_names.split('***')
names = [n.strip() for n in names]

PLAYER_ONE = names[0]
PLAYER_TWO = names[1]

print('\n')
print(f'Player one: {PLAYER_ONE}')
print(f'Player two: {PLAYER_TWO}')

prefix = f'{PLAYER_ONE} and {PLAYER_TWO} '
premise = prompt.open_question(
    question=(f'Given the setting, why are {PLAYER_ONE} and {PLAYER_TWO} about '
              'to interact?'),
    answer_prefix=prefix)
premise = f'{prefix}{premise}'

print('\n')
print(premise)

player_one_context = prompt.open_question(
    question=(f'{PLAYER_ONE} has a goal or interest that, if pursued, '
              f'would complicate things for {PLAYER_TWO}. What is it?'),
    max_tokens=1000,
)

print('\n')
print(player_one_context)

player_two_context = prompt.open_question(
    question=(f'{PLAYER_TWO} has a goal or interest that, if pursued, '
              f'would complicate things for {PLAYER_ONE}. What is it?'),
    max_tokens=1000,
)

print('\n')
print(player_two_context)


In [ ]:
# @title Configure instances.

instances = [
    prefab_lib.InstanceConfig(
        prefab='basic__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': PLAYER_ONE,
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='myagent__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': PLAYER_TWO,
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='generic__GameMaster',
        role=prefab_lib.Role.GAME_MASTER,
        params={
            'name': 'default rules',
            'acting_order': 'game_master_choice',
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='dialogic__GameMaster',
        role=prefab_lib.Role.GAME_MASTER,
        params={
            'name': 'conversation rules',
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='formative_memories_initializer__GameMaster',
        role=prefab_lib.Role.INITIALIZER,
        params={
            'name': 'initial setup rules',
            'next_game_master_name': 'default rules',
            'shared_memories': statements,
            'player_specific_context': {PLAYER_ONE : [player_one_context],
                                        PLAYER_TWO : [player_two_context]},
        },
    ),
]

In [ ]:
config = prefab_lib.Config(
    default_premise=premise,
    default_max_steps=5,
    prefabs=prefabs,
    instances=instances,
)

# The simulation

In [ ]:
# @title Initialize the simulation
runnable_simulation = simulation.Simulation(
    config=config,
    model=model,
    embedder=embedder,
)

In [ ]:
# @title Run the simulation
raw_log = []
results_log = runnable_simulation.play(max_steps=5,
                                       raw_log=raw_log)

In [ ]:
# @title Display the log
display.HTML(results_log)